# Siamese Networks
 Keras to implement a simple example of Siamese networks, which will verify whether two MNIST images are from the same class or not

Start with the import statements:

In [ ]:
import random
import numpy as np
import tensorflow as tf

Next, we'll implement the prepareData function to create the train/test dataset (both for training and testing):

In [ ]:
def prepareData(inputs: np.ndarray, labels: np.ndarray):
      classesNumbers = 10
      digitalIdx = [np.where(labels == i)[0] for i in range(classesNumbers)]
      pairs = list()
      labels = list()
      n = min([len(digitalIdx[d]) for d in range(classesNumbers)]) - 1
      for d in range(classesNumbers):
        for i in range(n):
            z1, z2 = digitalIdx[d][i], digitalIdx[d][i + 1]
            pairs += [[inputs[z1], inputs[z2]]]
            inc = random.randrange(1, classesNumbers)
            dn = (d + inc) % classesNumbers
            z1, z2 = digitalIdx[d][i], digitalIdx[dn][i]
            pairs += [[inputs[z1], inputs[z2]]]
            labels += [1, 0]
      return np.array(pairs), np.array(labels, dtype=np.float32)

Next, let's implement the createTemplate function, which defines one branch of the Siamese network:

In [ ]:
def createTemplate():
      return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.15),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.15),
        tf.keras.layers.Dense(64, activation='relu'),
        ])

Next, let's build the whole training system, starting from the MNIST dataset:

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]

11490434/11490434 [==============================] - 1s 0us/step


We'll use the raw dataset to create the actual train and test verification datasets:

In [ ]:
train_pairs, tr_labels = prepareData(x_train, y_train)
test_pairs, test_labels = prepareData(x_test, y_test)

Then, we'll build the base portion of the Siamese network:

In [ ]:
base_network = createTemplate()

Next, let's create the two branches:

In [ ]:
# Create first half of the siamese system
input_a = tf.keras.layers.Input(shape=input_shape)
# Note how we reuse the base_network in both halfs
enconder1 = base_network(input_a)
# Create the second half of the siamese system
input_b = tf.keras.layers.Input(shape=input_shape)
enconder2 = base_network(input_b)

Next, we'll create the measure of similarity, which uses the outputs of enconder1 and enconder2. It is implemented as a tf.keras.layers.Lambda layer:

In [ ]:
distance = tf.keras.layers.Lambda(
    lambda embeddings: tf.keras.backend.abs(embeddings[0] - embeddings[1])) \
    ([enconder1, enconder2])

Then, we'll create the final fully connected layer, which takes the output of the distance and compresses it to a single sigmoid output:

In [ ]:
measureOfSimilarity = tf.keras.layers.Dense(1, activation='sigmoid') (distance)

Finally, we can build the model and initiate the training for 10 epochs:

In [ ]:
# Build the model
model = tf.keras.models.Model([input_a, input_b], measureOfSimilarity)
# Train
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

model.fit([train_pairs[:, 0], train_pairs[:, 1]], tr_labels,
          batch_size=128,epochs=10,validation_data=([test_pairs[:, 0], test_pairs[:, 1]], test_labels))

Epoch 1/10
847/847 [==============================] - 17s 16ms/step - loss: 0.3535 - accuracy: 0.8475 - val_loss: 0.2727 - val_accuracy: 0.9097
Epoch 2/10
847/847 [==============================] - 9s 11ms/step - loss: 0.1845 - accuracy: 0.9314 - val_loss: 0.1970 - val_accuracy: 0.9374
Epoch 3/10
847/847 [==============================] - 9s 10ms/step - loss: 0.1251 - accuracy: 0.9552 - val_loss: 0.1395 - val_accuracy: 0.9570
Epoch 4/10
847/847 [==============================] - 8s 9ms/step - loss: 0.0951 - accuracy: 0.9661 - val_loss: 0.1184 - val_accuracy: 0.9644
Epoch 5/10
847/847 [==============================] - 9s 11ms/step - loss: 0.0781 - accuracy: 0.9725 - val_loss: 0.1033 - val_accuracy: 0.9698
Epoch 6/10
847/847 [==============================] - 9s 10ms/step - loss: 0.0656 - accuracy: 0.9772 - val_loss: 0.0954 - val_accuracy: 0.9718
Epoch 7/10
847/847 [==============================] - 8s 9ms/step - loss: 0.0563 - accuracy: 0.9802 - val_loss: 0.0884 - val_accuracy: 0.9730
